# Calculate Monthly Anomaly and Nino34 Index Using CDO


In this notebook we demonstrate how to calculate monthly anomoly and Nino34 index using CDO:

- Look at the information contained in our example file
- Visualise the data using ncview 
- Calculate monthly anomaly
- Calculate Nino34 Climate Index

This example uses the Coupled Model Intercomparison Project (CMIP6) collections. 

---
inspired by the notebook in https://github.com/NCI-data-analysis-platform/climate-cmip.git
- Original Authors: NCI Virtual Research Environment Team
- Keywords: CMIP, CDO, monthly anomaly, Nino34 Index
- Create Date: 2019-Oct; Update Date: 2021-Feb
---
- Adapted to DKRZ env: S. Kindermann: 2022-Aug

This notebook is licenced under the [Creative Commons Attribution 4.0 International license](https://creativecommons.org/licenses/by/4.0/)

## What is CDO?

CDO stands for "Climate Data Operators". CDO is a collection of command line operators to manipulate and analyse Climate and NWP model Data. It support the following data formats: GRIB 1/2, netCDF 3/4, SERVICE, EXTRA and IEG. There are more than 600 operators available. See [CDO's homepage](https://code.mpimet.mpg.de/projects/cdo) for more information about this library.

To load the CDO module on the interactive shell at DKRZ, run:

``` $ module load cdo```

In [ ]:
#!module load cdo

### List the path of the CMIP6 data

Here we will be using an example CMIP6 file that includes the near surface temperature from the 20th century all-forcing historical simulation based on the CESM2.0 model of NCAR. 

In [ ]:
! ls /pool/data/CMIP6/data/CMIP/NCAR/CESM2/historical/r1i1p1f1/Amon/tas/gn/v20190308/tas_Amon_CESM2_historical_r1i1p1f1_gn_185001-201412.nc

### Have a look at the data 

**CDO Basic usage:** 
`$ cdo sinfo filename | head -xx`

**head** displays only onely the first lines in the termial.

In [ ]:
!cdo sinfo /pool/data/CMIP6/data/CMIP/NCAR/CESM2/historical/r1i1p1f1/Amon/tas/gn/v20190308/tas_Amon_CESM2_historical_r1i1p1f1_gn_185001-201412.nc | head -15

### Let's see which years this file includes

We can use the function **showyear** to display all the years in this file

**Basic usage:**

```
$ cdo showyear <file>
```


In [ ]:
!cdo showyear /pool/data/CMIP6/data/CMIP/NCAR/CESM2/historical/r1i1p1f1/Amon/tas/gn/v20190308/tas_Amon_CESM2_historical_r1i1p1f1_gn_185001-201412.nc

### Calculate monthly anomaly 

In this exercise, we are going to calculate the monthly anomaly between 1971 and 2000. We will:

1. Choose climatology years 1971-2000 
2. Calculate the monthly average during 1971-2000
3. Subtract the monthly average from the original file to obtain the difference, i.e., anomaly.

We need three functions to implement this procedure:

 
```
$ cdo selyear, year_lower, year_upper <input.nc> <output.nc> 

```

``selyear`` selects the years of interest and generates a new file.


```
$ cdo ymonmean <input.nc> <output.nc> 

```

`ymonmean` calculates the mean monthly values from a multi-year monthly timeseries.


```
$ cdo ymonsub <input1.nc> <input2.nc> <output.nc> 

```

`ymonsub` calculates the difference between two input files and saves this difference as an output file.

However, if we do the three steps above we will generate unnecessary intermediate files. CDO allows for the piping of multiple commands so we can chain together the commands above:

```
$ cdo ymonsub <input.nc> -ymonmean -selyear,1971,2000 <input2.nc> output.nc

```


In [ ]:
### create output directory if it doesn't already exist
import os
outdir = './output'
if not os.path.exists(outdir):
    os.mkdir(outdir)
    
### run our cdo command
!cdo ymonsub /pool/data/CMIP6/data/CMIP/NCAR/CESM2/historical/r1i1p1f1/Amon/tas/gn/v20190308/tas_Amon_CESM2_historical_r1i1p1f1_gn_185001-201412.nc -ymonmean -selyear,1971/2000 /pool/data/CMIP6/data/CMIP/NCAR/CESM2/historical/r1i1p1f1/Amon/tas/gn/v20190308/tas_Amon_CESM2_historical_r1i1p1f1_gn_185001-201412.nc ./output/tas_Amon_CESM2_historical_r1i1p1f1_gn_197101-200012_anomaly.nc

### Let's have a look at the monthly anomaly

In [ ]:
!ncdump -h ./output/tas_Amon_CESM2_historical_r1i1p1f1_gn_197101-200012_anomaly.nc

### Calculate Nino34 Index using monthly anomaly

There are several indices used to monitor the tropical Pacific, all of which are based on SST anomalies averaged across a given region.  Usually the anomalies are computed relative to a base period of 30 years.  The Niño 3.4 index and the Oceanic Niño Index (ONI) are the most commonly used indices to define El Niño and La Niña events. See more about [sea surface tempreture indices](https://climatedataguide.ucar.edu/climate-data/nino-sst-indices-nino-12-3-34-4-oni-and-tni).

To calcuate Nino34 Index, we need to choose the Nino34 region with lat/lon bbox = (-170,-120,-5,5)

In [ ]:
!cdo sellonlatbox,-170,-120,-5,5 ./output/tas_Amon_CESM2_historical_r1i1p1f1_gn_197101-200012_anomaly.nc ./output/Nino34Region.nc

Then use function **fldavg** to calculate the field average and save the result as the Nino34 index.

In [ ]:
!cdo fldavg ./output/Nino34Region.nc ./output/Nino34Idx.nc

### Summary

In this example, we show how to use CDO command line operators to calculate monthly anomaly and Nino34 Climate Index.

## Reference


https://code.mpimet.mpg.de/projects/cdo/embedded/cdo.pdf